### Тестирование RAG

В этом ноутбуке прописан пайплайн прогона валидационной подвыборки через RAG и подсчета метрик Retrieval

In [1]:
import numpy as np
import requests
import os

In [2]:
path_to_articles = r"C:\Users\tre_i\Downloads\converted-20251213T172526Z-3-001\converted"

In [3]:
len(os.listdir(path_to_articles))

1000

Выберем 30 документов из 100

In [4]:
import random
import pandas as pd


#val_names = random.sample(os.listdir(path_to_articles), 30)
#df = pd.DataFrame(val_names)
#df.to_csv("val_names.csv", index=False)

val_names = pd.read_csv('val_names.csv')['0']

val_paths = [os.path.join(path_to_articles, el) for el in val_names]

In [5]:
%load_ext autoreload
%autoreload 2
from docker.test_api import (test_upload_dataset,
                             test_parse_documents,
                             test_retrieve,
                             test_generate)

In [6]:
meta_path = r'C:\Users\tre_i\PycharmProjects\ScienceRAG\data\raw'

In [7]:
from json import load

with open(os.path.join(meta_path, 'queries.json'), 'r', encoding="utf-8") as f:
    queries = load(f)
with open(os.path.join(meta_path, 'qrels.json'), 'r', encoding="utf-8") as f:
    qrels = load(f)
with open(os.path.join(meta_path, 'answers.json'), 'r', encoding="utf-8") as f:
    answers = load(f)

In [8]:
len(queries)

3045

In [9]:
val_ids = [k for k, v in qrels.items() if v['doc_id'] + '.txt' in set(val_names)]
len(val_ids)

96

In [10]:
docs_with_queries = set([v['doc_id'] + '.txt' for v in qrels.values() if v['doc_id'] + '.txt' in set(val_names)])
len(docs_with_queries)

12

In [11]:
val_queries = {k: v for k, v in queries.items() if k in val_ids}
len(val_queries)

96

То есть с этими 30 документами связаны 96 вопросов (относятся к 12 документам, для остальных вопросов в списке от создателей датасета нет)

In [12]:
os.mkdir('val_texts')

In [13]:
from shutil import copy


for val_path in val_paths:
    copy(val_path, 'val_texts')

In [14]:
%%time

upload_response = test_upload_dataset('val_texts')

CPU times: total: 15.6 ms
Wall time: 3.54 s


In [15]:
dataset_id = upload_response.json()['dataset_id']
dataset_id

'33940024d92511f0bf4e0242ac130006'

In [16]:
upload_response.json()

{'status': 'success', 'dataset_id': '33940024d92511f0bf4e0242ac130006'}

In [17]:
import dotenv


dotenv.load_dotenv('.env.host')

True

In [18]:
import requests

BASE_URL = os.getenv('RAGFLOW_BASE_URL')
API_KEY = os.getenv('RAGFLOW_API_KEY')
DATASET_ID = dataset_id

url = f"{BASE_URL}/api/v1/datasets/{DATASET_ID}/documents"

headers = {
    "Authorization": f"Bearer {API_KEY}"
}

resp = requests.get(url, headers=headers)
resp.raise_for_status()

data = resp.json()

documents = data["data"]["docs"]

ragflow_doc_ids = {doc["name"]: doc["id"] for doc in documents}

In [19]:
queries_ground_truth_docs = {}

for k in val_queries:
    qrels_doc = qrels[k]
    doc_id = qrels_doc['doc_id']
    doc_name = doc_id + '.txt'
    ragflow_doc_id = ragflow_doc_ids[doc_name]
    queries_ground_truth_docs[k] = ragflow_doc_id

In [20]:
%%time
test_parse_documents(dataset_id)

CPU times: total: 31.2 ms
Wall time: 2min 27s


<Response [200]>

In [21]:
from tqdm import tqdm


retrieve_responses = {}

for k, v in tqdm(val_queries.items()):
    query = v['query']
    response = test_retrieve(query, [dataset_id])
    retrieve_responses[k] = response.json()

100%|██████████| 96/96 [01:34<00:00,  1.01it/s]


In [22]:
retrieve_responses_doc_ids_ranked = {}
for k, v in retrieve_responses.items():
    document_ids_ranked = []
    for suggestion in v:
        document_ids_ranked.append(suggestion['document_id'])
    retrieve_responses_doc_ids_ranked[k] = document_ids_ranked

In [23]:
list(retrieve_responses.values())[0][0]

{'id': '5ab29eeb170feadb',
 'content': '\n\\end{aligned}$$where $p(u)$ is the probability of a random location in the image space and it has been simplified because it is the same in both source and target distributions, i.e., $p_{s}(u)=p_{t}(u)$. In theory $p_{t}(y)$ is unknown, thus Eq. 5 cannot be applied. Despite that, we found that setting $p_{t}(y)=\\frac{1}{C}$ works well, because it forces the model to do balanced detections on the test set. In practice, this maximises average precision because this metric independently evaluates all classes and it rewards balanced detectors [17]. Accordingly, the Classification (C) calibration of the logit $z_{y}$ is:$\\mathrm{C}\\left(z_{y}\\right)=\\left\\{\\begin{array}{l}z_{y}-\\log _{\\beta}\\left(\\frac{n_{y}}{\\sum_{i} n_{i}}\\right)+\\log _{\\beta}\\left(\\frac{1}{C}\\right), \\quad y \\in\\{1, \\ldots, C\\} \\\\ z_{y}, \\quad y=\\mathrm{bg},\\end{array}\\right.$where $\\beta$ is the base of the logarithm that we optimise through hyper

In [24]:
retrieve_responses_doc_ids_ranked

{'3a004357-19ab-4ed6-b60c-02e419657cc4': ['33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '33b98f92d92511f0bf4e0242ac130006',
  '3494f24

In [31]:
import numpy as np

def recall_at_k_single_value(ground_truth, prediction, k):
    return int(ground_truth in prediction[:k])

def mrr_at_k_single_value(ground_truth, prediction, k):
    for i, d in enumerate(prediction[:k], 1):
        if d == ground_truth:
            return 1 / i
    return 0

def precision_at_k_single_value(ground_truth, prediction, k):
    return int(ground_truth in prediction[:k]) / k

def retrieval_metric_at_k(ground_truths, predictions, metric, k):
    assert set(ground_truths.keys()) == set(predictions.keys())
    assert metric in {'recall', 'precision', 'mrr'}
    single_metric_values = []
    for query_id in ground_truths:
        ground_truth = ground_truths[query_id]
        prediction = predictions[query_id]
        if metric == 'recall':
            single_metric_value = recall_at_k_single_value(ground_truth, prediction, k)
        elif metric == 'mrr':
            single_metric_value = mrr_at_k_single_value(ground_truth, prediction, k)
        elif metric == 'precision':
            single_metric_value = precision_at_k_single_value(ground_truth, prediction, k)
        single_metric_values.append(single_metric_value)
    total_metric_value = np.mean(single_metric_values)
    return total_metric_value

In [40]:
retrieval_metric_at_k(
    ground_truths = queries_ground_truth_docs,
    predictions = retrieve_responses_doc_ids_ranked,
    metric = 'recall',
    k = 1
)

np.float64(0.9375)

In [32]:
ground_truths = queries_ground_truth_docs
predictions  = retrieve_responses_doc_ids_ranked

Ks = [1, 3, 5]
metrics = ["recall", "precision", "mrr"]

table = []
for k in Ks:
    row = {"k": k}
    for m in metrics:
        row[m] = retrieval_metric_at_k(ground_truths, predictions, m, k)
    table.append(row)

df = pd.DataFrame(table).set_index("k")
display(df.round(4))

,recall,precision,mrr
k,,,
1,0.9375,0.9375,0.9375
3,0.9896,0.3299,0.9635
5,0.9896,0.1979,0.9635


Получили высокие значения Top-k метрик для retrieval (precision для top-k скорее не имеет смысла)

Метрики качества генерации посчитаем с помощью LLM As Judge в другом ноутбуке

In [27]:
[len(set(el)) for el in retrieve_responses_doc_ids_ranked.values()]

[2,
 9,
 5,
 10,
 5,
 11,
 9,
 9,
 7,
 14,
 8,
 5,
 1,
 8,
 10,
 1,
 8,
 2,
 11,
 11,
 1,
 1,
 9,
 3,
 13,
 9,
 8,
 8,
 9,
 9,
 7,
 1,
 12,
 3,
 6,
 9,
 9,
 1,
 11,
 6,
 8,
 1,
 9,
 1,
 10,
 6,
 5,
 10,
 5,
 1,
 4,
 2,
 1,
 4,
 12,
 11,
 8,
 8,
 11,
 8,
 4,
 8,
 10,
 16,
 1,
 7,
 12,
 7,
 6,
 9,
 8,
 1,
 7,
 1,
 6,
 12,
 6,
 6,
 4,
 8,
 9,
 9,
 10,
 6,
 12,
 3,
 2,
 10,
 7,
 8,
 7,
 9,
 8,
 4,
 7,
 8]

In [33]:
generate_responses = {}


for k, v in tqdm(val_queries.items()):
    query = v['query']
    response = test_generate(query, [dataset_id])
    generate_responses[k] = response.json()

100%|██████████| 96/96 [04:37<00:00,  2.89s/it]


In [37]:
queries_ground_truth_answers = {}

for k in val_queries:
    queries_ground_truth_answers[k] = answers[k]

In [38]:
generate_responses_answers = {}
for k, v in generate_responses.items():
    answer = v['answer']
    generate_responses_answers[k] = answer

In [45]:
from json import dump

with open('val_retrieve_responses.json', 'w', encoding="utf-8") as f:
    dump(retrieve_responses, f)
with open('val_generate_answers.json', 'w', encoding="utf-8") as f:
    dump(generate_responses_answers, f)
with open('val_generate_ground_truths.json', 'w', encoding="utf-8") as f:
    dump(queries_ground_truth_answers, f)
with open('val_queries.json', 'w', encoding="utf-8") as f:
    dump(val_queries, f)